# Devoir de Machine Learning - Notebook : Batch Normalization et Batch Renormalization
## ISAE-SUPAERO SDD 2020/2021

### Ondongh-Essalt Guillaume 

Le but de ce Notebook est de présenter les concepts de Batch Normalization et Batch Renormalization dans le cadre du deep learning. Le premier de ces deux concepts, la Batch Normalization, fut à l'origine d'optimisations majeures de l'apprentissage des réseaux de neurones profonds. Elle a été formalisée pour la première fois en 2015 par Sergey Ioffe et Christian Szegedy, deux chercheurs chez Google, dans leur article "Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift" [1]. Suite à l'essort et la popularité de cette méthode dans les années qui ont suivit la publication, les premières critiques sont tombées. C'est notamment l'instabilité de la méthode face à un jeu de données d'entrainement biaisé lors de l'échantillonnage qui formera l'une de ses limites les plus importantes. Pour contrer cette faiblesse, Sergey Ioffe a proposé une extention à la Batch Normalization en 2017 dans son article "Batch Renormalization: Towards Reducing Minibatch Dependence in Batch-Normalized Models"[2].

In [ ]:
#Installations: TensorFlow, Keras
#!pip install tensorflow
#!pip install keras
#Imports :
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Batch Normalization

### Contexte et idée

Lors de l'apprentissage d'un réseau de neuronne profond, le jeu de données d'entrainement peut présenter pour chacune de ses dimension des distributions très variées. Des échelles de distribution trop distantes peuvent causer des problèmes lors de l'apprentissage car les entrées relativement trop élevées pourraient alors se propager à travers le réseau de neuronne et engendrer un gradient désequilibré voir une explosion de ce dernier. Il est alors intéressant de ramener toutes les données d'entrée à la même échelle en utilisant les procédés de **Normalisation** et de **Standardisation**. On emploit souvent une normalisation dite standard :



<img src="img/formule.png" width="900px" align=middle ></img>


L'avantage de cette méthode est qu'elle augmente drastiquement la vitesse d'aprentissage du réseau.

### Internal covariate shift

**Introduction au covariate shift :**

-> intuition : si ma distribution d'entrée change, il faut réentrainer d'avantage le modèle pour le généraliser.

Dans l'image ci-dessous, on cherche à illustrer le principe de covariate shift. On veut ici classifier les images qui présentent un chat de celles qui n'en présentent pas. Lors de la première instance d'apprentissage, on ne passe que des chats noirs en entrée du réseau. À la seconde instance, on passe des chats de formes et couleurs différentes. Le modèle, qui aura appris lors de la première instance, devra donc être corrigé davantage lors de l'apprentissage après le passage de la deuxième distribution. Le covariate shift est le fait que la distribution des données passées en entrée du réseau diffère d'une instance d'apprentissage à l'autre. On comprend alors que le modèle va devoir intégrer chaque nouvelle distribution successive en corrigeant ses paramètres d'apprentissage. Ceci aura pour conséquence de modifier la fonction de décision finale.

<img src="img/covariate.png" width="1200px"></img>

L'inconvénient de la normalisation simple présentée plus haut est que cette méthode se limite uniquement à la première couche. Celle-ci voit donc en entrée les valeurs transformées du jeu de données d'entraînement. Or, chaque couche successive de neurones peut rencontrer le même problème de distribution en entrée. En effet, après chaque mise à jour des poids d'entraînement, on passe de nouvelles données à l'entrées du réseau qui vont se propager.

<img src="img/reseau.png" width="900px"></img> <img src="img/reseau_2.png" width="900px"></img>

Il peut arriver que l'un des poids soit beaucoup plus élevé que les autres poids d'une même couche ou bien que les nouvelles donées passées présentent des valeurs extrêmes. Dans l'image ci-dessus, la 3ème couche voit ses valeurs d'entrée (a1 a2 a3 a4) changer à chaque instance. Or, rien ne garantit après plusieurs mises à jour des poids que ces valeurs soient elles aussi normales. En conséquence, ce déséquilibre à l'entrée de la couche cachée pourrait causer à son tour un déséquilibre de la distribution à la sortie de cette couche et donc engendrer les même instabilités citées précédement. On appelle ce phénomène "internal covariate shift" car il concerne les distributions perçues du point de vue de chacune des couches cachées du réseau.


### Algorithme

#### Entraînement

Pour contrer le phénomène d'internal covariate shift, l'idée de l'article de Sergey Ioffe et Christian Szegedy est d'appliquer à une couche donnée **une normalisation des sorties des fonctions d'activation**. Cette normalisation se fait **pour chaque dimension** sur l'ensemble d'un **batch**, d'où le terme de "Batch Normalization". Cette méthode garantit une distribution plus centrée et réduite des valeurs en sortie de la couche et donc à l'entrée de la couche suivante. En pratique, elle est plus coûteuse en calcul mais augmente considérablement la vitesse d'apprentissage. L'image ci-dessous explique plus en détails les opérations effectuées :

<img src="img/batchnorm.png" width="900px"></img> 

Lors de l'étape de normalisation, on rajoute un terme ε au dénominateur non nul mais très faible pour éviter les erreur de calcul numérique (division par 0).
Par ailleurs, on observera qu'une fois la normalisation effectuée, l'agorithme ne s'arrête pas là. Par la suite on vient multiplier la valeur normalisée xi chapeau par un paramètre γ et on vient lui additionner un paramètre β. Ces deux paramètres deviennent alors entraînables et permettent de rééchelonner et déplacer les valeurs pour optimiser l'apprentissage. L'apprentissage de ces paramètres se fait lors de la descente de grandient stochastique selon les règles de dérivation en chaîne explicitées ci-dessous.

<img src="img/chainrule.png" width="900px"></img>

#### Inférence

Nous avons vu que lors de l'apprentissage, la normalisation se faisait à l'aide des statistiques du batch. Or, dans le cas de mini-batch par exemple, cela implique que ces statistiques changent pour chaque instance d'apprentissage. En revanche, lors de l'inférence d'un élément, on aimerait que la sortie ne dépende que de l'entrée et ce de manière déterministe. Il convient donc de fixer une bonne fois pour toute les paramètres μ et σ avant de prédire. Pour cela, on utilise la forme suivante :

<img src="img/inference.png" width="600px"></img> 


E[x] est alors la moyenne effectuée sur l'ensemble des données d'entrainement et Var[x] est l'estimation de la variance : 

<img src="img/inference3.png" width="600px"></img>

où m est la taille d'un batch et l'espérance calculée sur l'ensemble des variances des batchs d'entraînement.

Enfin, on n'onbliera pas de multiplier par le paramètre γ et d'additionner le paramètre β, tous deux appris lors de l'entraînement 

<img src="img/inference2.png" width="600px"></img>

### Exemple

On génère des données circulaires avec un bruit gaussien en utilisant la fonction sklearn.datasets.make_circles(). Les deux classes sont données par les labels 0 et 1. Le but est alors d'apprendre à classer les points en fonction de leurs coordonnées. Pour cela on va construire un MLP.

In [ ]:
# scatter plot of the circles dataset with points colored by class
from sklearn.datasets import make_circles
from numpy import where

# generate circles
X, y = make_circles(n_samples=1000, noise=0.1, random_state=1)

# select indices of points with each class label
for i in range(2):
	samples_ix = where(y == i)
	plt.scatter(X[samples_ix, 0], X[samples_ix, 1], label=str(i))
plt.legend()
plt.show()

On décide de séparer nos 1000 points en 500 points d'entraînement et 500 points de test. 

In [ ]:
# split into train and test
n_train = 500
trainX, testX = X[:n_train, :], X[n_train:, :]
trainy, testy = y[:n_train], y[n_train:]

On crée ensuite notre modèle. Ce dernier prend en entrée les 2 coordonées des points (input_dim=2). On y incorpore une seule couche cachée dense de 50 neurones à fonction d'activation ReLU. Enfin on connecte cette couche à un neurone à fonction d'activation sigmoid.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# define model
model = Sequential()
model.add(Dense(50, input_dim=2, activation='relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation='sigmoid'))
opt = SGD(lr=0.01, momentum=0.9)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

# fit model
history = model.fit(trainX, trainy, validation_data=(testX, testy), epochs=100, verbose=0)

# evaluate the model
_, train_acc = model.evaluate(trainX, trainy, verbose=0)
_, test_acc = model.evaluate(testX, testy, verbose=0)
print('Train: %.3f, Test: %.3f' % (train_acc, test_acc))
# plot history
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.show()

On observe alors de bonnes performances : on dépasse le score de 80% au bout de 30 epochs. Essayons maintenant d'introduire une étape de batch normalization.

In [ ]:
# mlp for the two circles problem with batchnorm after activation function
from keras.layers import BatchNormalization

<div class="alert alert-success">
Exercise 1:
Create a new MLP model "model_2" identical to the model above including the keras BatchNormalization() function between the two dense layers
</div>

In [ ]:
#TO DO

In [41]:
# %load solutions/ex1.py

On observe bien de meilleurs temps d'apprentissage lorsqu'on introduit la Batch Normalization. Pour les données d'entraînement, on obtient un score au dessus de 80% en environ 20 epochs alors qu'on dépassait ce score autour de 30 epochs sans Batchnorm. 

### Régularisation : Conséquence de la batch normalization

Dans le cadre de mini-batch, le fait que les statistiques de normalisation soient calculées uniquement sur les données du mini-batch induit un certain bruit dans les sorties de la Batch Normalisation. Ce petit bruit, au même titre qu'un dropout, a un effet de régularisation dans le modèle. Cependant en pratique, cette régularisation ne vaut pas l'effet d'un dropout et il est déconseillé d'utiliser uniquement la Batch Normalization à cette fin.

## Limites de la Batch Normalization

Nous allons jouer un autre exemple basé sur le jeu de données iris de ScikitLearn. 

In [43]:
#Import Data
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()

In [44]:
# Load data into a DataFrame
df = pd.DataFrame(iris.data, columns=iris.feature_names)
# Convert datatype to float
df = df.astype(float)
# append "target" and name it "label"
df['label'] = iris.target
# Use string label instead
df['label'] = df.label.replace(dict(enumerate(iris.target_names)))

In [45]:
df.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

    label  
0  setosa  
1  setosa  
2  setosa  
3  setosa  
4  setosa

Les labels sont donnés de manière textuelle. Il convient de les modifier pour pouvoir les présenter au modèle. Pour cela, on passe par un encodage one-hot.

https://fr.wikipedia.org/wiki/Encodage_one-hot

In [46]:
# label -> one-hot encoding
label = pd.get_dummies(df['label'], prefix='label')
df = pd.concat([df, label], axis=1)
# drop old label
df.drop(['label'], axis=1, inplace=True)

In [47]:
df.head()

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.5                1.4               0.2   
1                4.9               3.0                1.4               0.2   
2                4.7               3.2                1.3               0.2   
3                4.6               3.1                1.5               0.2   
4                5.0               3.6                1.4               0.2   

   label_setosa  label_versicolor  label_virginica  
0             1                 0                0  
1             1                 0                0  
2             1                 0                0  
3             1                 0                0  
4             1                 0                0

In [ ]:
# Creating X and y
X = df[['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']]
# Convert DataFrame into np array
X = np.asarray(X)
y = df[['label_setosa', 'label_versicolor', 'label_virginica']]
# Convert DataFrame into np array
y = np.asarray(y)

In [ ]:
#Separate dataset between train and test
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.20, random_state=42)

Maintenant que les données sont prêtes à être traitées, il faut construire notre modèle. On prend bien 4 valeurs en entrée (input_shape=(4,)) et en sortie on retrouve les 3 neurones correspondant aux 3 labels avec une fonction d'activation softmax. Les sorties peuvent donc être considérées comme les probabilités d'appartenance à chaque classe.

In [ ]:
model_3 = Sequential([
    Dense(64, input_shape=(4,), activation="relu"),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')
]);

In [ ]:
model_3.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history = model_3.fit(
    X_train, 
    y_train, 
    epochs=200, 
    validation_split=0.25, 
    batch_size=40, 
    verbose=2)

In [ ]:
%matplotlib inline
def plot_metric(history, metric):
    train_metrics = history.history[metric]
    val_metrics = history.history['val_'+metric]
    epochs = range(1, len(train_metrics) + 1)
    plt.plot(epochs, train_metrics)
    plt.plot(epochs, val_metrics)
    plt.title('Training and validation '+ metric)
    plt.xlabel("Epochs")
    plt.ylabel(metric)
    plt.legend(["train_"+metric, 'val_'+metric])
    plt.show()

In [ ]:
plot_metric(history, 'accuracy')

In [ ]:
plot_metric(history, 'loss')

In [ ]:
# Evaluate the model on the test set
model_3.evaluate(X_test, y_test, verbose=2)

<div class="alert alert-success">
Exercise 2:
    
1. Create a new MLP model "model_4" identical to the model above including the keras BatchNormalization() function after each dense layer. 
    
2. We also invite you to play with the BatchNormalization() parameters : momentum, epsilon, beta_initializer, gamma_initializer. One can analyse the influence of eachon on the model results. For syntax, de BatchNormalization() at https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization or check the solution.
</div>

In [ ]:
#TO DO

In [42]:
# %load solutions/ex2.py

In [ ]:
model_4.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history = model_4.fit(
    X_train, 
    y_train, 
    epochs=200, 
    validation_split=0.25, 
    batch_size=40, 
    verbose=2)

In [ ]:
plot_metric(history, 'accuracy')

In [ ]:
plot_metric(history, 'loss')

In [ ]:
# Evaluate the model on the test set
model_3.evaluate(X_test, y_test, verbose=2)

### Résultats

On observe bien, comme dans l'exemple précédent, que la vitesse d'entraînement augmente lorsqu'on ajoute des étapes de Batch Normalization. En revanche, l'algorithme semble être plus lent pour prédire correctement les valeurs de validation. Pour expliquer ce phénomène, il faut comprendre la manière dont le modèle infère sur les données de validation. En effet, le fait que la moyenne et la variance utilisées pour faire des prédictions soient glissantes implique que le modèle est moins bon pour généraliser à de nouvelles valeurs avant d'avoir vu suffisamment de données. De manière générale, si ces statistiques venaient à être trop différentes entre les phases d'entraînement et les tests (à cause de biais d'échantillonage par exemple), le modèle pourrait ne pas fonctionner. 

### Minibatch dependence

La limite importante que rencontre la Batch Normalization est ce que l'on appelle la minibatch dependence. C'est d'ailleurs ce qu'expriment les premières phrases dans l'abstract de Sergey Ioffe dans son article de 2017 : 

"Batch Normalization is quite effective at accelerating and improving the training of deep models. However, its effectiveness diminishes when the training mini- batches are small, or do not consist of independent samples."

- **Lorsque les mini-batch sont trop petits, l'estimation de la moyenne et de la variance faite sur chaque batch devient moins précise. Ces imprécisions peuvent alors se propager en profondeur et déteriorer le modèle.**

- **Lorsque les mini-batch présentent des données non i.i.d. (ce qui arrive en pratique lorsqu'on échantillonne de manière biaisée le jeu de données), le modèle risque d'overfit sur les distributions d'entraînement. Cela risque donc de déteriorer le modèle lors des tests et ce d'autant plus que les distributions d'entraînement et de test diffèrent.**


## Batch Renormalization

Dans les deux cas cités ci-dessus, le problème sous-jacent est que les moyennes et les variances utilisées lors de l'entraînement peuvent être trop différentes de celles calculées sur l'ensemble des données. Pour tenter de corriger ce biais, Ioffe propose une extention à la Batch Normalisation. **L'idée géniale est d'introduire la moyenne et la variance glissante dans la phase d'entraînement pour s'assurer que les statistiques de chaque batch ne diffèrent pas trop de celles de l'ensemble des données**. En effectuant la transformation affine suivante, il garanti ainsi que la sortie sera "identique" lors des deux phases d'entraînement et de test: 

<img src="img/transformation.png" width="900px" align=middle ></img>

Si σ = E[σB] et μ = E[μB] alors E[r] = 1 et E[d] = 0. On constate d'ailleurs que cette transformation est identique à une batch normalization si on fixe r=1 et b=0. 


Il appelle alors cet algorithme la batch renormalization :


<img src="img/batchrenorm.png" width="900px" align=middle ></img>

N.B.: À chaque instance de calcul de gradient, les paramètres r et d seront pris comme constants (donc non entraînables).
Le nouveau paramètre introduit par cette extention est α qui est par défaut assez grand pour pouvoir apporter une correction suffisamment importante lors de l'apprentissage. 

En pratique on ne commence pas directement par utiliser batch renorm. À la place on commence par lancer l'entrainement avec la batch norm classique et on augmente peu à peu le taux d'utilisation de batch renorm.

<div class="alert alert-success">
Exercise 3:
    
Create a new MLP model "model_5" identical to model_4 including Batch Renormalization instead of Batch Normalization. To help you, you can check for the right parameter to change in the documentation https://www.tensorflow.org/api_docs/python/tf/keras/layers/BatchNormalization .
</div>

In [ ]:
#TO DO

In [48]:
# %load solutions/ex3.py

In [ ]:
model_5.compile(
    optimizer='adam', 
    loss='categorical_crossentropy', 
    metrics=['accuracy']
)

In [ ]:
history = model_5.fit(
    X_train, 
    y_train, 
    epochs=200, 
    validation_split=0.25, 
    batch_size=40, 
    verbose=2)

In [ ]:
plot_metric(history, 'accuracy')

In [ ]:
plot_metric(history, 'loss')

In [ ]:
# Evaluate the model on the test set
model_5.evaluate(X_test, y_test, verbose=2)

On obsrerve que le modèle gagne en généralisation sur les valeurs de validation sans perdre en rapidité par rapport à la version Batch Norm. Cette correction est donc efficace. 

## Conclusion

Bien que nous ayons expliqué lors dans ce notebook les avantages de la Batch Normalisation, nous avons vu qu'elle pouvait poser problème dans des cas spécifiques liés à l'echantillonnage de nos batchs. Ces problèmes sont en partie contrés par la Batch Renormalisation de Ioffe qui permet de trouver un compromis entre vitesse d'apprentissage et généralisation du modèle. Les exemples jouets du notebook excluent un autre cas d'utilisation de la Batch Normalization, celui des réseau convolutifs (voir BatchNorm2d() de pytorch utilisée dans le cours par exemple). L'application aux CNN a été un terrain fertile pour la publications sur les différentes approches pour normaliser les données issues des couches de convolution (voir Batch Norm, Layer Norm, Instance Norm, Group Norm dans l'article [3]). Cependant, les principes généraux necessaires à la compréhension de méthodes plus élaborées sont exposés ici.

## Bibliographie et références

Publications :

[1] Sergey Ioffe and Christian Szegedy. Batch normalization: Accelerating deep network training by reducing internal covariate shift.

[2] Sergey Ioffe. Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift.

[3] Yuxin Wu and Kaiming He. Group Normalization.

Articles en ligne :

[4] https://medium.com/luminovo/a-refresher-on-batch-re-normalization-5e0a1e902960

[5] https://www.alexirpan.com/2017/04/26/perils-batch-norm.html

Ressources pour les exemples :

[6] https://towardsdatascience.com/batch-normalization-in-practice-an-example-with-keras-and-tensorflow-2-0-b1ec28bde96f

[7] https://machinelearningmastery.com/how-to-accelerate-learning-of-deep-neural-networks-with-batch-normalization/

Autres ressources :

[8] Jerome H. Friedman, Robert Tibshirani et Trevor Hastie. The Elements of Statistical Learning: Data Mining, Inference, and Prediction.

[9] Aurélien Géron. Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow: Concepts, Tools, and Techniques to Build Intelligent Systems
